##  Zillow Economics Data (1996-2017) 
## Data Exploration and Cleanup

We utilized two datasets published by Zillow’s Economic Research Team (city_time_series and cities_crosswalk) containing housing and economic data from 1996 to 2017, made available on Kaggle - https://www.kaggle.com/zillow/zecon/data

* The city_time_series had over 4 million records, so it had to be uploaded to Jupyter notebook in sets of 500,000 records, using the ‘chunksize’ argument. The cities_crosswalk file contained location information,  broken down into City, County and State in separate columns. 
* The city_time_series dataset was reduced by selecting 13 columns of potential interest and by truncating the dataset to 2007-2017, which corresponds to the data range available from HUD homeless dataset.
* Both files were merged based on the RegionName/Unique_City_ID column.
* City and state were combined in one column ‘CityState’ and set as the index.
* We created a function to split the ‘Date’ column and create a ‘Year’ column. 
* A new dataframe was created containing 18 major cities of interest selected based on data available from the HUD homeless dataset. The new dataframe was grouped by ‘CityState’ and ‘Year’ columns, to get the mean median house price per year, and was exported as a CSV file ‘18_zillow_selected_cities.csv’, and pushed to Github for further analysis. 
* NaNs were not removed in order to visually assess which columns had the most/least gaps in data for future analysis. 

In [15]:
#Import dependencies
import csv
import pandas as pd
from pandas import *
from IPython.display import display

In [16]:
#Read and assess the file
data_chunks = read_csv("City_time_series.csv", iterator=True, chunksize=500000)
city_prices_df= concat(data_chunks, ignore_index=True)
#display(city_prices_df.head())

city_geo_df= read_csv("cities_crosswalk.csv")
city_geo_df= city_geo_df.rename(columns = {'Unique_City_ID' : 'RegionName'})
#display(city_geo_df.head())

In [17]:
#Output anatomy of files
print ('Total # of rows in city_prices_df: %s' % (len(city_prices_df)))
print ('Total # of columns in city_prices_df: %s' % len(city_prices_df.columns))
print ('Total # of rows in city_geo_df: %s' % (len(city_geo_df)))
print ('Total # of columns in city_geo_df: %s' % len(city_geo_df.columns))
#print(list(city_geo_df))

Total # of rows in city_prices_df: 4172721
Total # of columns in city_prices_df: 85
Total # of rows in city_geo_df: 25341
Total # of columns in city_geo_df: 4


In [18]:
#Create a new city_prices_df with only the columns of interest -ADDED 4ADDITIONAL COLUMNS
Zillow_df = city_prices_df[["Date","RegionName","MedianSoldPricePerSqft_AllHomes","MedianSoldPricePerSqft_SingleFamilyResidence",
               "MedianSoldPrice_AllHomes","ZHVIPerSqft_AllHomes","ZHVI_AllHomes", "ZHVI_SingleFamilyResidence", 
               "MedianRentalPricePerSqft_AllHomes", "MedianRentalPricePerSqft_SingleFamilyResidence"]]             

#Merge the two dataframes
merged_df = pd.merge(city_geo_df, Zillow_df, on="RegionName")

#Checkout merged_df output
print ('Total # of rows in df: %s' % (len(merged_df)))
print ('Total # of columns in df: %s' % len(merged_df.columns))

Total # of rows in df: 4172721
Total # of columns in df: 13


In [19]:
#Set date as index
mod_merged_df=merged_df.set_index('Date')

#Remove data prior to 2007
ten_years_merged_df= mod_merged_df.sort_index().truncate(before='2007-01-31')

In [ ]:
#Combine city and state in a new column to call later
new_column = ten_years_merged_df['City'] + ", " + ten_years_merged_df['State']
ten_years_merged_df.insert(loc=0, column='CityState',value= new_column)

#Remove unecessary columns
final_zillow_df = ten_years_merged_df.drop(['RegionName', 'City', 'County', 'State'], axis=1)

#Reset index to City
reset_final_zillow_df=final_zillow_df.reset_index().set_index('CityState') 

In [ ]:
def ext_year(date):
    year = date.split('-')
    return year[0]

reset_final_zillow_df["Year"] = reset_final_zillow_df["Date"].apply(ext_year)
reset_final_zillow_df = reset_final_zillow_df.drop(['Date'], axis=1)
display(reset_final_zillow_df)

In [17]:
#Selected cities below are based on cross-examination of data available for cities in HUD dataset
Cities = ["New York, NY", "Philadelphia, PA", "Boston, MA", "Washington, DC", "Chicago, IL", 
          "Minneapolis, MN", "Denver, CO", "Salt Lake City, UT", "Seattle, WA", "Los Angeles, CA", 
          "San Francisco, CA","Miami, FL,","Charlotte, NC", "Atlanta, GA", "Detroit, MI", 
          "Anchorage, AK", "Honolulu, HI", "Indianapolis, IN"]

Selected_cities= (reset_final_zillow_df.loc[Cities])

#Groupby CityState and Year and get mean
grouped_CityState_Year= Selected_cities.groupby(["CityState", "Year"])
Zillow_selected_cities_mean= grouped_CityState_Year.mean()
display (Zillow_selected_cities_mean)

#Print csv file
Zillow_selected_cities_mean.to_csv("18_zillow_selected_cities.csv")

MedianSoldPricePerSqft_AllHomes  \
CityState         Year                                    
Anchorage, AK     2007                              NaN   
                  2008                              NaN   
                  2009                              NaN   
                  2010                              NaN   
                  2011                              NaN   
                  2012                              NaN   
                  2013                              NaN   
                  2014                              NaN   
                  2015                              NaN   
                  2016                              NaN   
                  2017                              NaN   
Atlanta, GA       2007                       168.931508   
                  2008                       154.565250   
                  2009                       145.401150   
                  2010                       146.298967   
                  2011                       153.547900   
                  2012                       150.160933   
                  2013                       167.450092   
                  2014                       182.251608   
                  2015                       194.083075   
                  2016                       206.584386   
                  2017                              NaN   
Boston, MA        2007                       355.376100   
                  2008                       361.741483   
                  2009                       338.552000   
                  2010                       360.332650   
                  2011                       381.571258   
                  2012                       382.170042   
                  2013                       427.719958   
                  2014                       459.340508   
...                                                 ...   
San Francisco, CA 2010                       571.629117   
                  2011                       546.432183   
                  2012                       603.421683   
                  2013                       720.226483   
                  2014                       834.283592   
                  2015                       949.884517   
                  2016                       950.291514   
                  2017                              NaN   
Seattle, WA       2007                       339.065017   
                  2008                       316.641600   
                  2009                       285.674525   
                  2010                       288.759733   
                  2011                       270.015833   
                  2012                       266.292525   
                  2013                       295.080875   
                  2014                       318.234317   
                  2015                       359.184458   
                  2016                       417.986400   
                  2017                              NaN   
Washington, DC    2007                       397.184017   
                  2008                       402.398867   
                  2009                       403.229425   
                  2010                       414.455117   
                  2011                       412.378150   
                  2012                       418.741275   
                  2013                       456.611067   
                  2014                       472.769608   
                  2015                       498.196842   
                  2016                       507.444371   
                  2017                              NaN   

                        MedianSoldPricePerSqft_SingleFamilyResidence  \
CityState         Year                                                 
Anchorage, AK     2007                                           NaN   
                  2008                                           NaN   
                  